# <center> <img src="../labs/img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
## <center> **Procesamiento de Datos Masivos** </center>
---
### <center> **Primavera 2025** </center>
---
### <center> **Ejemplos de Aprendizaje Automático (Machine Learning): K-means** </center>

---
**Profesor**: Dr. Pablo Camarillo Ramirez

In [10]:
import findspark
findspark.init()

#### Creacion de la conexión con el cluster de spark


In [11]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("MLSpark-K-means") \
    .master("spark://f5db43ce3d38:7077") \
    .config("spark.ui.port","4040") \
    .getOrCreate()
sc = spark.sparkContext
spark.conf.set("spark.sql.shuffle.partitions", "5")

# Preparación de Datos

In [12]:
from team_name.spark_utils import SparkUtils

# Sample data in Python (e.g., 2D points)
data = [
    (0, 1.0, 1.0),
    (1, 2.0, 1.0),
    (2, 4.0, 5.0),
    (3, 5.0, 5.0),
    (4, 10.0, 10.0),
    (5, 12.0, 11.0)
]

# Define schema for the DataFrame
schema = SparkUtils.generate_schema([("id", "integer"), ("x", "float"), ("y", "float")])

# Create DataFrame for k means
random_points_df = spark.createDataFrame(data, schema)

### Assemble the features into a single vector column

In [13]:
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(inputCols=["x", "y"], outputCol="features")
assembled_df = assembler.transform(random_points_df)

# Initialize KMeans

In [14]:
from pyspark.ml.clustering import KMeans

kmeans = KMeans().setK(2).setSeed(15)

# TRAINNING

In [15]:
model = kmeans.fit(assembled_df)

# PREDICTIONS

In [16]:
predictions = model.transform(assembled_df)

# EVALUATE MODEL

In [17]:
from pyspark.ml.evaluation import ClusteringEvaluator

# Evaluate clustering by computing Silhouette score
evaluator = ClusteringEvaluator()
silhouette = evaluator.evaluate(predictions)
print(f"Silhouette score: {silhouette}")

# Show the result
print("Cluster Centers: ")
for center in model.clusterCenters():
    print(center)

Silhouette score: 0.8754012705075422
Cluster Centers: 
[3. 3.]
[11.  10.5]


----------------------------------------
Exception occurred during processing of request from ('127.0.0.1', 39316)
Traceback (most recent call last):
  File "/opt/conda/lib/python3.11/socketserver.py", line 317, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/opt/conda/lib/python3.11/socketserver.py", line 348, in process_request
    self.finish_request(request, client_address)
  File "/opt/conda/lib/python3.11/socketserver.py", line 361, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/opt/conda/lib/python3.11/socketserver.py", line 755, in __init__
    self.handle()
  File "/opt/conda/spark-3.5.4-bin-hadoop3-scala2.13/python/pyspark/accumulators.py", line 295, in handle
    poll(accum_updates)
  File "/opt/conda/spark-3.5.4-bin-hadoop3-scala2.13/python/pyspark/accumulators.py", line 267, in poll
    if self.rfile in r and func():
                           ^^^^^^
  File "/opt/conda/spark-3.5.4-bin-hadoop3-sca

In [9]:
sc.stop()